# 과적합, 분산 편향 트레이드오프, 교차 검증

### 일반화 오차: 분산 + 편향^2 + 노이즈 
### 분산 편향 트레이드오프란: 분산과 편향은 반비례의 관계가 있으므로 이를 잘 맞추어서 모델링을 해야함

In [25]:
#필요한 라이브러리 임포트

import numpy as np
import pandas as pd

# 교차검증

## 1. 사이킷런의 model_selection의 KFold()를 사용하는 경우(For loop 사용)

#### 폴드를 분리할 객체 생성

In [1]:
from sklearn.model_selection import KFold

#### 데이터를 준비하고 회귀 모형 객체를 생성

In [12]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
diab = load_diabetes()

X=diab.data
y=diab.target
Lr = LinearRegression()

In [5]:
X.shape, y.shape

((442, 10), (442,))

#### split()함수를 호출하여 폴드별로 분리될 행 인덱스 세트를 구함

In [9]:
kfold = KFold(5)
list(kfold.split(X))[0]

(array([ 89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
        102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
        115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
        167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
        180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
        193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205,
        206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
        219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231,
        232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244,
        245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257,
        258, 259, 260, 261, 262, 263, 264, 265, 266

In [13]:
from sklearn.metrics import r2_score
r2_scores = []
for train_idx, test_idx in kfold.split(X):
    X_train = X[train_idx]
    X_test = X[test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]
    
    reg = Lr.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

In [30]:
#r2_scores # 이 다섯개의 오차의 평균이 예측 오차
for i,r2 in enumerate(r2_scores):
    print(i+1, f' - R2 = {r2:.3f}')
print(f' average R2 = {np.mean(r2_scores)}')

1  - R2 = 0.430
2  - R2 = 0.523
3  - R2 = 0.483
4  - R2 = 0.426
5  - R2 = 0.550
 average R2 = 0.48231643590864215


## 2. 사이킷런의 cross_val_score 함수를 사용하여 K폴드 교차 검증 수행 without shuffling:
- for loop 필요 없음

In [32]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

diab = load_diabetes()
X= diab.data
y= diab.target

Lr = LinearRegression()

print(cross_val_score(Lr, X, y, cv = 5),np.mean(cross_val_score(Lr, X, y, cv = 5)))

[0.42955615 0.52259939 0.48268054 0.42649776 0.55024834] 0.48231643590864215


In [37]:
# 여기는 cross_val_score 하면 안됨! 왜냐하면 분류라서 셔플을 한 다음에 나누어줘야함
from sklearn.datasets import load_iris
#import pandas as pd
iris = load_iris()
X=iris.data
y=iris.target
iris_df = pd.DataFrame(X, columns = iris.feature_names)
iris_df['Kind'] = iris.target
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Kind
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## 3. 사이킷런의 cross_val_score 함수를 사용하여 K폴드 교차 검증 수행 with shuffling

In [39]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import random as rd

iris = load_iris()
X= iris.data
y= iris.target
iris_df = pd.DataFrame(X, columns = iris.feature_names)
iris_df['Kind'] = iris.target

kfold = KFold(5, shuffle=True, random_state=29)
Lr = LinearRegression()


print(cross_val_score(Lr, X, y, cv = kfold),np.mean(cross_val_score(Lr, X, y, cv = kfold)))

[0.84997133 0.95733172 0.9182143  0.93251334 0.9361105 ] 0.9188282376059309
